In [3]:
import cv2
import numpy as np

# 에지 검출과 응용
- 에지는 한쪽방향으로 픽셀값이 급격히 바뀌는 부분임
- 어두운 영역에서 갑자기 밝아지거나 이의 반대의 경우 에지라고 함
- 경계 , 객체의 윤곽을 알아낼수있는 유용한 방법
- 에지를 찾으려면 픽셀값의 변화율이 큰 픽셀을 택해야함
- 그래서 미분 사용 



- 영상에서 미분 시 고려할점 
    1. 영상은 2차원 평면에서 정의되어 있는 함수임
    2. 영상은 정수 단위 좌표에 픽셀이 나열되어 있는 이산함수임
    - 뭐 여튼 x y 축 둘다의 미분값을 봐야한다  

## 편미분

In [ ]:
# x 축 미분 , y 축 미분 보기 
src = cv2.imread('./data/lenna.bmp',cv2.IMREAD_GRAYSCALE)  

Mx = np.array([[-1, 0,1],
                [-2, 0,2],
                [-1,0,1]], dtype =np.float32)

My = np.array([[-1,-2, -1],
                [0,0,0],
                [1,2,1]], dtype=np.float32)


dx = cv2.filter2D(src, -1 ,Mx, delta=128) 
dy = cv2.filter2D(src, -1, My, delta=128
) 


cv2.imshow('src', src) 
cv2.imshow('dx', dx) 
cv2.imshow('dy', dy)  

cv2.waitKey()
cv2.destroyAllWindows()


## 그래디언트 
    - 2차원 함수에서 x y 축 미분을 한꺼번에 벡터로 표현한것4    
    - 벡터의 뱡향은 변화 정도가 가장 큰 방향을 나타냄( x or y) 
    - 벡터의 크기는 변화율의 세기를 나타내는 척도임 
    - 2차원 영상에서 에지를 찾는 기본은 그래디언트 크기가 특정 값보다 큰 위치를 찾는거임

## 마스크 기반 에지 검출 - 소벨 마스크
    - cv2.Sobel() : 이 안에 3x3 마스크가 잇음  


In [ ]:
src = cv2.imread('./data/lenna.bmp', cv2.IMREAD_GRAYSCALE) 

dx = cv2.Sobel(src, cv2.CV_32FC1, 1,0) # x축 미분
dy = cv2.Sobel(src, cv2.CV_32FC1, 0,1) # y축 미분 

# cv2.Sobel 경우 cv2.filter2D() 랑 마스크 연산까진 같은데 자료형변환은 안되어있음
# 포화연산도 안되어었음 (음수값이 들어있다) 

# x축 미분 + y축 미분 = 그래디언트, magnitude함수가 그래디언트의 크기를 구해줌 
fmag= cv2.magnitude(dx, dy) 
# 포화연산 + 자료형 변환
mag = np.clip(fmag, 0, 255).astype(np.uint8)

# 영상 이진화 
ret, dst = cv2.threshold(mag, 160, 255, cv2.THRESH_BINARY)

cv2.imshow('mag',mag)
cv2.imshow('src', src)
cv2.imshow('dst', dst)  
cv2.waitKey()
cv2.destroyAllWindows()

## 캐니 에지 검출기
    * 소벨 에지 검출 방법의 단점을 해결
    * 검출기의 조건!!
        - 정확한 검출 : 정확하게 에지만 검출해야함
        - 정확한 위치: 실제에지의 중심을 찾아야함
        - 단일 에지: 하나의 에지는 하나의 점으로 표현되어야함

In [8]:
src =cv2.imread('./data/lenna.bmp', cv2.IMREAD_GRAYSCALE) 

# 캐니 에지 
dst = cv2.Canny(src , 80, 160) # low : high = (1:2 or 1:3) 이 제일 좋음

cv2.imshow('src', src) 
cv2.imshow('dst', dst) 
cv2.waitKey()
cv2.destroyAllWindows()

## 허프 변환 직선 검출
    - 에지 중에서도 직선만을 찾기 위한 기법
    - 에지를 찾아내고 , 에지 픽셀들이 일직선상에 배열되어있는지 확인
    - 2차원 xy 좌표에서 직선의 방정식을 파마리터 공간으로 변환하여 직선을 찾는 알고리즘 

In [1]:
import math

In [4]:
src = cv2.imread('./data/building.jpg', cv2.IMREAD_GRAYSCALE)

# 캐니 먼저 
edge = cv2.Canny(src, 100, 200) 

# 허프변환 직선 검출
rho= 1 # 숫자가 작을수로 직선이 더 정밀하게 검출됨 근데 연산시간이 걸림
theta = math.pi/180 
threshold = 160 # 축적 배열의 숫자가 높다는 것은 직선을 이루는 점들이 많다는 뜻
                # 얼마나 큰 값을 직선으로 판단할지는 threshold 에 달려있음

minLineLength =50  # 검출할 선분의 최소길이 
maxLineGap = 5 # 직선으로 간주할 에지 점 최대간격

lines = cv2.HoughLinesP(edge, rho , theta, threshold, 
                        minLineLength=minLineLength, maxLineGap= maxLineGap)

# 직선을 그릴 도화지 (3차원 도화지) 
dst = cv2.cvtColor(edge, cv2.COLOR_GRAY2BGR)

if lines is not None : 
    for i in range(len(lines)) : 
        line =lines[i][0]
        pt1 = line[0], line[1]
        pt2 = line[2], line[3]
        cv2.line(dst, pt1, pt2, (0,0,255), 2, cv2.LINE_AA) 
        

cv2.imshow('dst', dst) 
cv2.imshow('edge', edge)
cv2.imshow('src', src) 
cv2.waitKey()
cv2.destroyAllWindows()